In [2]:
from unstructured.partition.pdf import partition_pdf
raw_pdf_elements=partition_pdf(
    filename=r"C:\\Users\\ishan\\Documents\\Personal Study\\RAG\\mrr\\RefineDNet_A_Weakly_Supervised_Refinement_Framework_for_Single_Image_Dehazing.pdf",                  # mandatory
    strategy="hi_res",                                 # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                       # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir="extracted_data",  # optional - only works when ``extract_image_block_to_payload=False``
    )

In [3]:
raw_pdf_elements

In [4]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))





In [5]:
img=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))

In [6]:
tab=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

In [7]:
import os

path = os.listdir("extracted_data")

for i in range(len(path)):
    path[i] = "extracted_data/" + path[i]

In [8]:
import re

def extract_numeric_part(s):
    return int(re.findall(r'\d+', s)[0])

path = sorted(path, key=extract_numeric_part)

In [9]:
# filter if table keyword is present
path = [i for i in path if "table" not in i.lower()]

In [10]:
imag_summ=[]
image_encode_base64 = []


## Image

In [ ]:
import base64
import os
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
for i in range(len(path)):
    image_encode_base64.append(encode_image(path[i]))

In [15]:
import requests

def send_image_and_text_to_api(image_path, Title, img_text, api_url):
    # Open the image file
    with open(image_path, 'rb') as image_file:
        # Create a dictionary with the file and text to send
        files = {'query_image': ('image.jpg', image_file, 'image/jpeg')}
        data = {'Title': Title, 'img_text': img_text}
        
        # Send a POST request to the API
        response = requests.post(api_url, files=files, data=data)
    
    # Return the response from the API
    return response.json()

# Usage
api_url = 'http://localhost:3000/api/image/generate'  
# image_path = 'example_image.jpg' 
# Title = "Image Dehazing"
# img_text = "Image of a foggy landscape"
img_text = Title

# for i in range(3):
for i in range(len(path)):
    result = send_image_and_text_to_api(path[i], Title, img_text, api_url)
    # image_encode_base64.append(result['summary'].split(''))
    img_text.append(result['summary'].split('@@'))

# result = send_image_and_text_to_api(image_path, Title, img_text, api_url)
# print(result)


## Table

In [20]:
import requests


tab_summ = []
def send_text_to_api(query, api_url):
    # Create a dictionary with the query
    data = {'query': query}
    
    # Send a POST request to the API
    response = requests.post(api_url, json=data)
    
    # Return the response from the API
    return response.json()

# Usage
api_url = 'http://localhost:3000/api/text/generate'  # Replace with your actual API URL
# query = question

# for i in range(len(tab)):
for i in range(3):
    question = f"""<|user|>\n<image>You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table:{tab[i]} Note: two time @ is just for indexing without any meaning @@<|end|>\n<|assistant|>\n"""
    result = send_text_to_api(tab[i], api_url)
    tab_summ.append(result['summary'])

# result = send_text_to_api(query, api_url)
# print(result)


In [ ]:
tab_summ

## Text

In [25]:
import requests

text_summaries = []

def send_text_to_api(query, api_url):
    # Create a dictionary with the query
    data = {'query': query}
    
    # Send a POST request to the API
    response = requests.post(api_url, json=data)
    
    # Return the response from the API
    return response.json()

# Usage
api_url = 'http://localhost:3000/api/text/generate'  # Replace with your actual API URL
query = question

# for i in range(len(NarrativeText)):
for i in range(3):
    question = f"""<|user|>\n<image>You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the text that is well optimized for retrieval. Text:{NarrativeText[i]} Note: two time @ is just for indexing without any meaning @@<|end|>\n<|assistant|>\n"""
    result = send_text_to_api(NarrativeText[i], api_url)
    text_summaries.append(result['summary'])

# result = send_text_to_api(query, api_url)
# print(result)


## RAG

In [ ]:
import requests

def send_summaries_2_rag(text_summaries, NarrativeText, tab, tab_summ, image_encode_base64, imag_summ, api_url):
    # Create a dictionary with the query
    data = {
            'text_summaries': text_summaries,
            'Text': NarrativeText,
            'tab': tab,
            'tab_summ': tab_summ,
            'image_encode_base64': image_encode_base64,
            'imag_summ': imag_summ
            }
    
    # Send a POST request to the API
    response = requests.post(api_url, json=data)
    
    # Return the response from the API
    return response.json()

api_url = 'http://localhost:3000/api/rag/rag'  # Replace with your actual API URL

result = send_summaries_2_rag(text_summaries, NarrativeText, tab, tab_summ, image_encode_base64, imag_summ, api_url)

## Inference RAG - Query

In [ ]:
question = "What is the summary of the text?"

def send_query_to_rag(question, api_url):
    # Create a dictionary with the query
    data = {'query': question}
    
    # Send a POST request to the API
    response = requests.post(api_url, json=data)
    
    # Return the response from the API
    return response.json()

api_url = 'http://localhost:3000/api/rag/query'  # Replace with your actual API URL


result = send_query_to_rag(question, api_url)